In [130]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from json import loads
from requests import get
import re
from langdetect import detect

In [131]:
# connect to the database

dotenv_path = os.path.join("..", "scraping", ".env")
load_dotenv(dotenv_path)
DATABASE_URL = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"

engine = create_engine(DATABASE_URL)

query = "SELECT * FROM job_postings WHERE description IS NOT NULL;"
original_df = pd.read_sql(query, engine)
df = original_df.copy()

query2 =  """
    SELECT job_sources.*
    FROM job_sources 
    INNER JOIN job_postings AS j 
    ON job_sources.job_id = j.job_id
    WHERE j.description IS NOT NULL;
"""
original_df2 = pd.read_sql(query2, engine)
df2 = original_df2.copy()

In [132]:
df.head()

,id,job_id,title,location,salary,experience_level,job_type,employment_type,description,responsibilities,requirements,skills,benefits,company,description_criteria,language,description_en,title_en
0,6538,4158196588,Rituals - Wholesale Intern (Warsaw),"Warsaw, Mazowieckie, Poland (On-site)",None,None,None,None,"W dziale Wholesale & Travel Retail dbamy o to,...",None,None,None,None,Rituals Cosmetics Polska,Seniority level Internship Employment type Int...,pl,"""In the Wholesale section. ""As well as externa...","""Rituals - Wholelesale Intern (Warsaw)"""
1,361380,4186936564,"Full-stack Developer (React, Node.js) / Projec...","Bydgoszcz, Kujawsko-pomorskie, Poland (Remote)",None,None,None,None,Cześć👋 Jesteśmy 50+ osobowym zespołem pragmaty...,None,None,None,None,Exlabs,Seniority level Mid-Senior level Employment ty...,pl,"""Hi👋 We are a 50-person team of pragmatics who...","""Full-Stack Developer (React, Node. JS) / Proj..."
2,298526,4188240566,Główna Księgowa/Główny Księgowy,"Tychy, Śląskie, Poland (On-site)",None,None,None,None,"What We Offer: At Magna, you can expect an en...",None,None,None,None,Magna International,Seniority level Mid-Senior level Employment ty...,pl,"""What in Offer: AT Magna, You Can Expect An En...","""Chief accountant/chief accountant"""
3,275327,4187444908,Programista MS Dynamics 365BC/NAV,"Cracow, Małopolskie, Poland (Hybrid)",None,None,None,None,Nasz klient to renomowana firma z ponad 20-let...,None,None,None,None,ipracujzdalnie.pl,Seniority level Entry level Employment type Fu...,pl,"""Our client is a renowned company with over 20...","""MS Dynamics 365BC/NAV programmer"""
4,348481,4153864061,Mid/Senior Python Developer,"Łódź, Łódzkie, Poland (Remote)",None,None,None,None,Description & Requirements: Kogo szukamy: Doł...,None,None,None,None,Deloitte,Seniority level Mid-Senior level Employment ty...,pl,"""Description. ""Including a solid experience in...","""Mid/Senior Python Developer"""


In [133]:
df2.head()

,id,job_id,source,job_url,date_posted,is_active
0,1,4158353862,LinkedIn,https://www.linkedin.com/jobs/view/4158353862,2025-02-21,True
1,2,4138407547,LinkedIn,https://www.linkedin.com/jobs/view/4138407547,2025-02-21,True
2,3,4144364769,LinkedIn,https://www.linkedin.com/jobs/view/4144364769,2025-02-21,True
3,4,4083706042,LinkedIn,https://www.linkedin.com/jobs/view/4083706042,2025-02-21,True
4,5,4154765012,LinkedIn,https://www.linkedin.com/jobs/view/4154765012,2025-02-21,True


In [134]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

In [135]:
def update_description(text):
    sentences = re.split(r'(?<=[a-ząęśćźżół])(?=[A-ZĄĘŚĆŹŻÓŁ])', text)
    return ", ".join(sentences)

df["description"] = df["description"].apply(update_description)

In [136]:


def split_text(text, max_length=450):
    """Splits text into chunks, ensuring each is under the max_length."""
    sentences = re.split(r'([.,;])', text)
    chunks = []
    temp_chunk = ""

    for sentence in sentences:
        if temp_chunk and len(temp_chunk) + len(sentence) + 1 > max_length:
            chunks.append(temp_chunk.strip())
            temp_chunk = sentence
        else:
            temp_chunk = f"{temp_chunk} {sentence}".strip() if temp_chunk else sentence
    if temp_chunk:
        chunks.append(temp_chunk.strip())
    
    return chunks


def translate_text(text):
    """Translates text using the free Google API."""
    url = f'https://translate.googleapis.com/translate_a/single?client=gtx&dt=t&sl=pl&tl=en&q="{str(text)}"'
    try:
        response = get(url)
        translated_json = loads(response.text)
        return translated_json[0][0][0] if translated_json else text
    except Exception as e:
        print(f"Error translating: {e}")
        return text

def translate_full_text(text):
    """Splits, translates, and reconstructs the translated text."""
    chunks = split_text(text)
    translated_chunks = [translate_text(chunk) for chunk in chunks]
    return ". ".join(translated_chunks)

In [137]:
from tqdm import tqdm
from sqlalchemy.exc import OperationalError
import time
import sys
import math
tqdm.pandas()

with engine.connect() as connection:
    connection.execute(text("ALTER TABLE job_postings ADD COLUMN IF NOT EXISTS description_en TEXT;"))
    connection.execute(text("ALTER TABLE job_postings ADD COLUMN IF NOT EXISTS language VARCHAR(10);"))

    connection.commit()

df = pd.read_sql("SELECT * FROM job_postings WHERE description IS NOT NULL;", engine)
df["language"] = df["language"].fillna('')


missing_language_rows = df[df["language"] == ""]
if not missing_language_rows.empty:
    print(f"Detecting language for {len(missing_language_rows)} descriptions...")
    df.loc[df["language"] == "", "language"] = df.loc[df["language"] == "", "description"].apply(detect_language)

    with engine.begin() as conn:
        for _, row in df[df["language"] != ""].iterrows():
            conn.execute(
                text("UPDATE job_postings SET language = :language WHERE job_id = :job_id"),
                {"language": row["language"], "job_id": row["job_id"]}
            )

df["description_en"] = df.get("description_en", pd.Series([None]*len(df)))

df_to_translate = df[(df["language"] == "pl") & (df["description_en"].isnull())].copy()
total = len(df_to_translate)
print(f"Total Polish descriptions to translate: {total}")

skip = False
if total < 5:
    print("No Polish descriptions to translate.")
    skip = True

if not skip:
    chunk_size = math.ceil(total / 25)
    
    for i in range(0, total, chunk_size):
        chunk = df_to_translate.iloc[i:i+chunk_size].copy()
        print(f"\n🔸 Translating chunk {i//chunk_size + 1}/25")
        time.sleep(3)
        chunk["translated_description"] = chunk["description"].progress_apply(translate_full_text)
    
        # Save to DB
        for _, row in chunk.iterrows():
            try:
                with engine.begin() as conn:
                    conn.execute(
                        text("UPDATE job_postings SET description_en = :desc_en WHERE job_id = :job_id"),
                        {"desc_en": row["translated_description"], "job_id": row["job_id"]}
                    )
            except OperationalError as e:
                print(f"OperationalError for job_id={row['job_id']}: {e}")
                time.sleep(3)
    
        print(f"Chunk {i//chunk_size + 1} saved to database.")


Total Polish descriptions to translate: 0
No Polish descriptions to translate.


In [138]:
df["final_description"] = df.apply(lambda row: row["description_en"] if pd.notnull(row["description_en"]) else row["description"], axis=1)

In [139]:
df.head()

,id,job_id,title,location,salary,experience_level,job_type,employment_type,description,responsibilities,requirements,skills,benefits,company,description_criteria,language,description_en,title_en,final_description
0,6538,4158196588,Rituals - Wholesale Intern (Warsaw),"Warsaw, Mazowieckie, Poland (On-site)",None,None,None,None,"W dziale Wholesale & Travel Retail dbamy o to,...",None,None,None,None,Rituals Cosmetics Polska,Seniority level Internship Employment type Int...,pl,"""In the Wholesale section. ""As well as externa...","""Rituals - Wholelesale Intern (Warsaw)""","""In the Wholesale section. ""As well as externa..."
1,361380,4186936564,"Full-stack Developer (React, Node.js) / Projec...","Bydgoszcz, Kujawsko-pomorskie, Poland (Remote)",None,None,None,None,Cześć👋 Jesteśmy 50+ osobowym zespołem pragmaty...,None,None,None,None,Exlabs,Seniority level Mid-Senior level Employment ty...,pl,"""Hi👋 We are a 50-person team of pragmatics who...","""Full-Stack Developer (React, Node. JS) / Proj...","""Hi👋 We are a 50-person team of pragmatics who..."
2,298526,4188240566,Główna Księgowa/Główny Księgowy,"Tychy, Śląskie, Poland (On-site)",None,None,None,None,"What We Offer: At Magna, you can expect an en...",None,None,None,None,Magna International,Seniority level Mid-Senior level Employment ty...,pl,"""What in Offer: AT Magna, You Can Expect An En...","""Chief accountant/chief accountant""","""What in Offer: AT Magna, You Can Expect An En..."
3,275327,4187444908,Programista MS Dynamics 365BC/NAV,"Cracow, Małopolskie, Poland (Hybrid)",None,None,None,None,Nasz klient to renomowana firma z ponad 20-let...,None,None,None,None,ipracujzdalnie.pl,Seniority level Entry level Employment type Fu...,pl,"""Our client is a renowned company with over 20...","""MS Dynamics 365BC/NAV programmer""","""Our client is a renowned company with over 20..."
4,348481,4153864061,Mid/Senior Python Developer,"Łódź, Łódzkie, Poland (Remote)",None,None,None,None,Description & Requirements: Kogo szukamy: Doł...,None,None,None,None,Deloitte,Seniority level Mid-Senior level Employment ty...,pl,"""Description. ""Including a solid experience in...","""Mid/Senior Python Developer""","""Description. ""Including a solid experience in..."


In [140]:
df["final_description"].iloc[0]

'"In the Wholesale section. "As well as external partners. Wholesale Intern will help the team in the areas of: data management, communication, analyzes and support of Back Office. If you want to learn more about sales both off - and online,". "This is an ideal internship for you! Your tasks: data management and their analysis based on reports in the Excel program - the basis of your activities arartly preparing files with data - tracking inputs and going out to the customer, updating basic data and support for planning inquiries. Ordering marketing materials from agencies for 7 markets.". "Taking care of the database - collecting information from Account Managers and Marketing Department. Creating monthly marketing reports. Support in management and orders management, daily control/marketing control on our partners\' pages. Updating documents related to wholesale sales. Inspire knowledge and skills! Your profile: You have a ground knowledge and skills in the field of Excel programs . 

In [141]:
df[df["description_criteria"].isnull()].index


Index([ 2183,  5292, 13942, 14471, 14477, 19765, 19964, 20402, 27594, 27686,
       28026, 29146],
      dtype='int64')

In [142]:
df.iloc[4913]


id                                                                   9208
job_id                                                         4148877033
title                                          Product Operations Analyst
location                             Warsaw, Mazowieckie, Poland (Hybrid)
salary                                                               None
experience_level                                                     None
job_type                                                             None
employment_type                                                      None
description              CBRE Advisory (CBRE sp. z o. o.) is a Polish ...
responsibilities                                                     None
requirements                                                         None
skills                                                               None
benefits                                                             None
company                               

In [143]:
df.drop(index=df[df["description_criteria"].isnull()].index, inplace=True)
df.iloc[4913]

id                                                                  20371
job_id                                                         4078042354
title                   Senior Cloud Solutions Integrations Engineer, ...
location                            Warsaw, Mazowieckie, Poland (On-site)
salary                                                               None
experience_level                                                     None
job_type                                                             None
employment_type                                                      None
description              Job Description: We are seeking for a highly ...
responsibilities                                                     None
requirements                                                         None
skills                                                               None
benefits                                                             None
company                               

In [144]:
# Extracting job type
def extract_job_type(text):
    result = []
    if "remote" in text.lower():
        result.append("Remote")
    elif "hybrid" in text.lower():
        result.append("Hybrid")
    elif "on-site" in text.lower() or "office" in text.lower():
        result.append("On-site")
    return ", ".join(result) if result else None


df["job_type"] = df["location"].apply(extract_job_type)
df["job_type"].value_counts()

job_type
Hybrid     9553
Remote     8981
On-site    8612
Name: count, dtype: int64

In [145]:
print(df["job_type"].isnull().sum())

df["final_description"][df["job_type"].isnull()].apply(extract_job_type).value_counts()


2395


final_description
Remote     768
Hybrid     535
On-site    475
Name: count, dtype: int64

In [146]:
df["job_type"] = df["job_type"].fillna(df["final_description"].apply(extract_job_type))
print(df["job_type"].isnull().sum())


617


In [147]:
def extract_employment_type(text):
    result = []
    if "contract" in text.lower():
        result.append("Contract")
    if "full-time" in text.lower():
        result.append("Full-time")
    if "part-time" in text.lower():
        result.append("Part-time")
    if "b2b" in text.lower():
        result.append("B2B")
    
    return ", ".join(result) if result else None

df["employment_type"] = df["description_criteria"].apply(extract_employment_type)
df["employment_type"].value_counts()


employment_type
Full-time              25685
Contract                2515
Part-time                412
Contract, Full-time        2
Name: count, dtype: int64

In [148]:
print(df["employment_type"].isnull().sum())

df["final_description"][df["employment_type"].isnull()].apply(extract_employment_type).value_counts()

927


final_description
Contract                          176
Contract, Full-time                43
Full-time                          33
Contract, B2B                      14
Part-time                           8
B2B                                 7
Contract, Part-time                 6
Full-time, Part-time                6
Contract, Full-time, Part-time      5
Name: count, dtype: int64

In [149]:
df["employment_type"] = df["employment_type"].fillna(df["final_description"].apply(extract_employment_type))

print(df["employment_type"].isnull().sum())

629


In [150]:
df[df["language"]=="pl"].head(5)

,id,job_id,title,location,salary,experience_level,job_type,employment_type,description,responsibilities,requirements,skills,benefits,company,description_criteria,language,description_en,title_en,final_description
0,6538,4158196588,Rituals - Wholesale Intern (Warsaw),"Warsaw, Mazowieckie, Poland (On-site)",None,None,On-site,None,"W dziale Wholesale & Travel Retail dbamy o to,...",None,None,None,None,Rituals Cosmetics Polska,Seniority level Internship Employment type Int...,pl,"""In the Wholesale section. ""As well as externa...","""Rituals - Wholelesale Intern (Warsaw)""","""In the Wholesale section. ""As well as externa..."
1,361380,4186936564,"Full-stack Developer (React, Node.js) / Projec...","Bydgoszcz, Kujawsko-pomorskie, Poland (Remote)",None,None,Remote,Full-time,Cześć👋 Jesteśmy 50+ osobowym zespołem pragmaty...,None,None,None,None,Exlabs,Seniority level Mid-Senior level Employment ty...,pl,"""Hi👋 We are a 50-person team of pragmatics who...","""Full-Stack Developer (React, Node. JS) / Proj...","""Hi👋 We are a 50-person team of pragmatics who..."
2,298526,4188240566,Główna Księgowa/Główny Księgowy,"Tychy, Śląskie, Poland (On-site)",None,None,On-site,Full-time,"What We Offer: At Magna, you can expect an en...",None,None,None,None,Magna International,Seniority level Mid-Senior level Employment ty...,pl,"""What in Offer: AT Magna, You Can Expect An En...","""Chief accountant/chief accountant""","""What in Offer: AT Magna, You Can Expect An En..."
3,275327,4187444908,Programista MS Dynamics 365BC/NAV,"Cracow, Małopolskie, Poland (Hybrid)",None,None,Hybrid,Full-time,Nasz klient to renomowana firma z ponad 20-let...,None,None,None,None,ipracujzdalnie.pl,Seniority level Entry level Employment type Fu...,pl,"""Our client is a renowned company with over 20...","""MS Dynamics 365BC/NAV programmer""","""Our client is a renowned company with over 20..."
4,348481,4153864061,Mid/Senior Python Developer,"Łódź, Łódzkie, Poland (Remote)",None,None,Remote,Full-time,Description & Requirements: Kogo szukamy: Doł...,None,None,None,None,Deloitte,Seniority level Mid-Senior level Employment ty...,pl,"""Description. ""Including a solid experience in...","""Mid/Senior Python Developer""","""Description. ""Including a solid experience in..."


In [151]:
# Count how many jobs don't contain both job_type and employment_type
missing_job_type_and_employment_type = df[df["job_type"].isnull() & df["employment_type"].isnull()]
print(f"Number of jobs without both job_type and employment_type: {len(missing_job_type_and_employment_type)}")


Number of jobs without both job_type and employment_type: 3


In [152]:
def clean_description(text):
    text = text.replace('"', '')
    text = text.strip()
    return text

df["final_description"] = df["final_description"].apply(clean_description)

In [153]:
df.drop(columns=["description", "description_en"], inplace=True)
df.columns

Index(['id', 'job_id', 'title', 'location', 'salary', 'experience_level',
       'job_type', 'employment_type', 'responsibilities', 'requirements',
       'skills', 'benefits', 'company', 'description_criteria', 'language',
       'title_en', 'final_description'],
      dtype='object')

In [154]:
with engine.connect() as connection:
    connection.execute(text("ALTER TABLE job_postings ADD COLUMN IF NOT EXISTS title_en TEXT;"))
    connection.execute(text("ALTER TABLE job_postings ADD COLUMN IF NOT EXISTS language VARCHAR(10);"))
    connection.commit()

df = pd.read_sql("SELECT * FROM job_postings WHERE title IS NOT NULL;", engine)
df["language"] = df["language"].fillna('')

missing_language_rows = df[df["language"] == ""]
if not missing_language_rows.empty:
    print(f"Detecting language for {len(missing_language_rows)} titles...")
    df.loc[df["language"] == "", "language"] = df.loc[df["language"] == "", "title"].apply(detect_language)

    with engine.begin() as conn:
        for _, row in df[df["language"] != ""].iterrows():
            conn.execute(
                text("UPDATE job_postings SET language = :language WHERE job_id = :job_id"),
                {"language": row["language"], "job_id": row["job_id"]}
            )

df["title_en"] = df.get("title_en", pd.Series([None]*len(df)))
df_to_translate = df[(df["language"] == "pl") & (df["title_en"].isnull())].copy()
total = len(df_to_translate)
print(f"Total Polish titles to translate: {total}")

skip = False
if total < 5:
    print("No Polish titles to translate.")
    skip = True

if not skip:
    chunk_size = math.ceil(total / 10)
    
    for i in range(0, total, chunk_size):
        chunk = df_to_translate.iloc[i:i+chunk_size].copy()
        print(f"\n🔸 Translating chunk {i//chunk_size + 1}/10")
        time.sleep(3)
        chunk["translated_title"] = chunk["title"].progress_apply(translate_full_text)
    
        for _, row in chunk.iterrows():
            try:
                with engine.begin() as conn:
                    conn.execute(
                        text("UPDATE job_postings SET title_en = :title_en WHERE job_id = :job_id"),
                        {"title_en": row["translated_title"], "job_id": row["job_id"]}
                    )
            except OperationalError as e:
                print(f"OperationalError for job_id={row['job_id']}: {e}")
                time.sleep(3)
            
        print(f"Chunk {i//chunk_size + 1} saved to database.")

    print(f"All titles translated and saved to database.")


Total Polish titles to translate: 0
No Polish titles to translate.


In [155]:
from google import genai

dotenv_path = os.path.join("..", "scraping", ".env")
load_dotenv(dotenv_path)

gemini_key = os.getenv("GEMINI_API_KEY")

client = genai.Client(api_key=gemini_key)


response = client.models.generate_content(
    model="gemini-2.0-flash", contents="From the job description extract data: responsibilities, requirements, skills, benefits, experience level (for example Internship, Junior, Mid, Senior, Director etc), salary if mentioned: Senior DevOps Engineer (Kubernetes Python Groovy)  For our client 1Dea we are looking for a person for the position of Senior Devops Engineer (Kubernetes Python Groovy). For one of our business partners, we are looking for an experienced Devops engineer:. We offer work in agile (agile) methodology, in an international environment where the language of communication is English. We do not require experience in the financial industry - everything is important, we will teach us all, we will teach us experience in the financial industry . We value communication skills because the work is of a team character. We offer stable employment with an 8 -hour work day, without duty and work at night. Conditions of commitment: area: finance / banking: 100% Remotor: ASAP (we accept candidates with max 3MSC period of notice) Size (set individually): 180 - 195 PLN Net VAT / hz. . Long-term recruitment process (100% remotely) :: A short information and information call with a 1DEA recruiter (~ 10 min) Technical and design verification with the client (~ 1-1, 5 h) (if OK) familiarization interview with the head of the client department (~ 30 min) decision to cooperate . Developmental environment and design infrastructure in lower environments. Creating and maintaining scripts in Shell and Python. Creating and maintaining streams CI/CD (groovy). Development of configuration and automation (mainly with the use of ANSIB). Development of microservices based on containers (docker and kubernetes). . Sharing knowledge about the Devops approach and solutions (internal training, demonstrations, presentations). Support for developers and engineers QA.. Requirements of a minimum of 5 years of commercial experience in a similar role (Devops Engineer) experience with Kubernetsdobra Knowledge of Jenkins or other CE/CDSIMENTS programming in Python and Groovydobra Knowledge of CM tools (we use Ansible) good knowledge of Linux and the ability to write script.. Optimization and maintenance of platform. Finding the GIT version control system (we use Bitbucket) Good communication skills communicative at B2 level: experience in MLOPS, HADOOP, SPARK, SONARQUBE, POSTGRESQ. . Software and configuration. The ability to work during flexible hours of remote work 100%professional consulting and support in career development from an experienced team of specialists 1DECE we will be friendly, openness, respect, mutual help and support in developing the competences of both our own as well as colleagues and colleagues from teams.. Each team member has the opportunity to propose his own ideas and solutions, and his voice is always taken into account"
)
print(response.text)


Here's a breakdown of the job description extract:

*   **Job Title:** Senior DevOps Engineer (Kubernetes Python Groovy)
*   **Responsibilities:**
    *   Development environment and design infrastructure in lower environments.
    *   Creating and maintaining scripts in Shell and Python.
    *   Creating and maintaining CI/CD pipelines (Groovy).
    *   Development of configuration and automation (mainly with Ansible).
    *   Development of microservices based on containers (Docker and Kubernetes).
    *   Sharing knowledge about the DevOps approach and solutions (internal training, demonstrations, presentations).
    *   Support for developers and QA engineers.
    *   Optimization and maintenance of the platform.

*   **Requirements:**
    *   Minimum of 5 years of commercial experience in a similar DevOps Engineer role.
    *   Experience with Kubernetes.
    *   Good knowledge of Jenkins or other CI/CD tools.
    *   Proficiency in Python and Groovy programming.
    *   Good know